In [10]:
import sys
#Change path specificly to your directories
sys.path.insert(1, '/home/fishial/Fishial/Object-Detection-Model')

import time
import torch
import os
import cv2
import math 
import matplotlib.pyplot as plt
import torchvision.models as models
import numpy as np
from torch import nn
from torch.optim import Optimizer, SGD
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from sklearn.neighbors import KDTree
import pandas as pd
from module.classification_package.src.utils import read_json, save_json, classify_by_database
from module.classification_package.interpreter_classifier import ClassifierFC
from module.classification_package.interpreter_embeding import EmbeddingClassifier
# from module.classification_package.interpreter_embeding_data import EmbeddingClassifierData
from module.segmentation_package.interpreter_segm import SegmentationInference
from module.classification_package.src.dataset import FishialDataset
# from module.segmentation_package.src.utils import resize_image
from PIL import Image
import numpy as np
import random
import sklearn.metrics.pairwise
import scipy.spatial.distance
import copy
import json
import time
import requests
import warnings
warnings.filterwarnings('ignore')

# import FiftyOne
import fiftyone as fo
import fiftyone.zoo as foz

from module.segmentation_package.interpreter_segm import SegmentationInference
import fiftyone.brain as fob


from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from os import listdir
from module.classification_package.src.utils import read_json, save_json
from os.path import isfile, join

In [21]:
def get_class_and_id(idx_data, ann_id):
    for i in idx_data:
        for k_id, k in enumerate(idx_data[i]['annotation_id']):
            if k == ann_id:
                return i,k_id

In [ ]:
dataset_full = fo.load_dataset("fish-classification-184")
selected_data = dataset_full.match_tags(['val', 'train'])

In [26]:
embedding_tensor = torch.load('/home/fishial/Fishial/output/classification/resnet_18_184_train_27_11/rect_full_embedding_dep_fixed.pt')
idx_data = read_json('/home/fishial/Fishial/output/classification/resnet_18_184_train_27_11/rect_full_idx_dep_fixed.json')
labels = read_json('/home/fishial/Fishial/output/classification/resnet_18_184_train_27_11/rect_full_labels_dep_fixed.json')
labels_reverse = {labels[label]: label for label in labels}

In [33]:
embeddings_list = []
for idx, sample in enumerate(selected_data):
    print(f"Left : {idx}/{len(selected_data)}", end='\r')
    ann_id = sample['annotation_id']
    class_id, val_id = get_class_and_id(idx_data, ann_id)
    embedding = embedding_tensor[int(class_id)][int(val_id)]
    embeddings_list.append(embedding)
    output = classify_by_database(embedding_tensor, embedding)
    sample['predictions'] = labels_reverse[str(class_id)]
    sample['predictions_distance'] = output[0][1].item()
    sample.save()

In [34]:
results = fob.compute_visualization(
    selected_data,
    embeddings=embeddings_list,
    num_dims=2,
    method="umap",
    brain_key="mnist_test",
    verbose=True,
    seed=51,)

Generating visualization...
UMAP(random_state=51, verbose=True)
Thu Dec  1 13:55:44 2022 Construct fuzzy simplicial set
Thu Dec  1 13:55:44 2022 Finding Nearest Neighbors
Thu Dec  1 13:55:44 2022 Building RP forest with 14 trees
Thu Dec  1 13:55:44 2022 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	Stopping threshold met -- exiting after 3 iterations
Thu Dec  1 13:55:50 2022 Finished Nearest Neighbor Search
Thu Dec  1 13:55:51 2022 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

Thu Dec  1 13:56:02 2022 Finished embedding


In [36]:
# Plot embeddings colored by ground truth label
plot = results.visualize(labels="polyline.label")
plot.show(height=720)

# Attach plot to session
session.plots.attach(plot)

FigureWidget({
    'data': [{'customdata': array(['6380bc22ccb46092e99addc0', '6380bc22ccb46092e99addc2',
    …

In [35]:
selected_data

Dataset:     fish-classification-184
Media type:  image
Num samples: 30326
Tags:        ['odm_false', 'odm_true', 'train', 'val']
Sample fields:
    id:                   fiftyone.core.fields.ObjectIdField
    filepath:             fiftyone.core.fields.StringField
    tags:                 fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:             fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    polyline:             fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Polyline)
    annotation_id:        fiftyone.core.fields.StringField
    image_id:             fiftyone.core.fields.StringField
    width:                fiftyone.core.fields.IntField
    height:               fiftyone.core.fields.IntField
    area_conf:            fiftyone.core.fields.FloatField
    drawn_fish_id:        fiftyone.core.fields.IntField
    predictions:          fiftyone.core.fields.StringField
    predictions_distance: fiftyone.core